In [1]:
import datetime as dt
from pathlib import Path
import os

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
data_path = Path(os.getcwd()+'/data/yoochoose/') 
train_path = data_path / 'ratings.dat'

def load_data(data_path: Path, nrows=None):
    data = pd.read_csv(data_path, sep='::', header=None, usecols=[0, 1, 2, 3], dtype={0: np.int32, 1: np.int32, 2: np.int32}, nrows=nrows)
    data.columns = ['UserId', 'ItemId', 'Rating', 'Time']
    return data

data = load_data(train_path, None)
data.sort_values(['UserId', 'Time'], inplace=True)  # data를 id와 시간 순서로 정렬해줍니다.
data

,UserId,ItemId,Rating,Time
31,1,3186,4,978300019
22,1,1270,5,978300055
27,1,1721,4,978300055
37,1,1022,5,978300055
24,1,2340,3,978300103
...,...,...,...,...
1000019,6040,2917,4,997454429
999988,6040,1921,4,997454464
1000172,6040,1784,3,997454464
1000167,6040,161,3,997454486


In [4]:
data['UserId'].nunique(), data['ItemId'].nunique()

(6040, 3706)

In [5]:
session_length = data.groupby('UserId').size()
session_length

UserId
1        53
2       129
3        51
4        21
5       198
       ... 
6036    888
6037    202
6038     20
6039    123
6040    341
Length: 6040, dtype: int64

In [6]:
session_length.min(), session_length.max()
#이상한 길이가있음.

(20, 2314)

In [8]:
session_length.describe()

count    6040.000000
mean      165.597517
std       192.747029
min        20.000000
25%        44.000000
50%        96.000000
75%       208.000000
max      2314.000000
dtype: float64

# IQR로 이상치 제거

In [9]:

q3 = session_length.quantile(0.75)
q1 = session_length.quantile(0.25)
upper_boundary = q3+3/2*(q3-q1)
upper_boundary

454.0

In [10]:
drop_index = session_length[session_length>upper_boundary].index
data.drop(index=drop_index, inplace=True)

In [11]:
oldest, latest = data['Time'].min(), data['Time'].max()
print(oldest) 
print(latest)

data['Time'] = pd.to_datetime(data['Time'], unit ='s')
data.head

956703932
1046454590


<bound method NDFrame.head of          UserId  ItemId  Rating                Time
31            1    3186       4 2000-12-31 22:00:19
22            1    1270       5 2000-12-31 22:00:55
27            1    1721       4 2000-12-31 22:00:55
37            1    1022       5 2000-12-31 22:00:55
24            1    2340       3 2000-12-31 22:01:43
...         ...     ...     ...                 ...
1000019    6040    2917       4 2001-08-10 14:40:29
999988     6040    1921       4 2001-08-10 14:41:04
1000172    6040    1784       3 2001-08-10 14:41:04
1000167    6040     161       3 2001-08-10 14:41:26
1000042    6040    1221       4 2001-08-20 13:44:15

[999734 rows x 4 columns]>

In [12]:
#1년간의 데이터만 사용
oldest, latest = data['Time'].min(), data['Time'].max()
print(oldest) 
print(latest)

2000-04-25 23:05:32
2003-02-28 17:49:50


In [18]:
data = data[data['Time'] > '2002-02-28']   # 방금 구한 날짜 이후의 데이터만 모은다. 

In [19]:
data.head()

,UserId,ItemId,Rating,Time
5170,36,1387,5,2002-03-12 03:46:59
5267,36,1201,4,2002-03-12 03:46:59
5122,36,1291,5,2002-03-12 03:47:16
5123,36,2167,5,2002-03-12 03:48:25
5290,36,2951,4,2002-03-12 03:48:25


# 데이터처리 

In [20]:
# short_session을 제거한 다음 unpopular item을 제거하면 다시 길이가 1인 session이 생길 수 있습니다.
# 이를 위해 반복문을 통해 지속적으로 제거 합니다.
def cleanse_recursive(data: pd.DataFrame, shortest, least_click) -> pd.DataFrame:
    while True:
        before_len = len(data)
        data = cleanse_short_session(data, shortest)
        data = cleanse_unpopular_item(data, least_click)
        after_len = len(data)
        if before_len == after_len:
            break
    return data


def cleanse_short_session(data: pd.DataFrame, shortest):
    session_len = data.groupby('UserId').size()
    session_use = session_len[session_len >= shortest].index
    data = data[data['UserId'].isin(session_use)]
    return data


def cleanse_unpopular_item(data: pd.DataFrame, least_click):
    item_popular = data.groupby('ItemId').size()
    item_use = item_popular[item_popular >= least_click].index
    data = data[data['ItemId'].isin(item_use)]
    return data

In [21]:
data = cleanse_recursive(data, shortest=2, least_click=5)
data

,UserId,ItemId,Rating,Time
5170,36,1387,5,2002-03-12 03:46:59
5267,36,1201,4,2002-03-12 03:46:59
5122,36,1291,5,2002-03-12 03:47:16
5123,36,2167,5,2002-03-12 03:48:25
5290,36,2951,4,2002-03-12 03:48:25
...,...,...,...,...
992358,5996,3835,3,2002-04-29 20:46:24
992279,5996,2422,3,2002-04-29 20:47:05
992702,5996,168,3,2002-09-03 13:12:26
992459,5996,339,4,2002-10-07 13:24:39


# 트레인셋 분리

In [22]:
def split_by_date(data: pd.DataFrame, n_days: int):
    final_time = data['Time'].max()
    session_last_time = data.groupby('UserId')['Time'].max()
    session_in_train = session_last_time[session_last_time < final_time - dt.timedelta(n_days)].index
    session_in_test = session_last_time[session_last_time >= final_time - dt.timedelta(n_days)].index

    before_date = data[data['UserId'].isin(session_in_train)]
    after_date = data[data['UserId'].isin(session_in_test)]
    after_date = after_date[after_date['ItemId'].isin(before_date['ItemId'])]
    return before_date, after_date

tr, test = split_by_date(data, n_days=60)
tr, val = split_by_date(tr, n_days=60)

In [23]:
# data에 대한 정보를 살펴봅니다.
def stats_info(data: pd.DataFrame, status: str):
    print(f'* {status} Set Stats Info\n'
          f'\t Events: {len(data)}\n'
          f'\t Users: {data["UserId"].nunique()}\n'
          f'\t Items: {data["ItemId"].nunique()}\n'
          f'\t First Time : {data["Time"].min()}\n'
          f'\t Last Time : {data["Time"].max()}\n')

In [24]:
stats_info(tr, 'train')
stats_info(val, 'valid')
stats_info(test, 'test')

* train Set Stats Info
	 Events: 6570
	 Users: 198
	 Items: 1463
	 First Time : 2002-02-28 04:35:32
	 Last Time : 2002-10-29 23:24:51

* valid Set Stats Info
	 Events: 3201
	 Users: 91
	 Items: 1229
	 First Time : 2002-03-02 00:43:37
	 Last Time : 2002-12-30 02:26:14

* test Set Stats Info
	 Events: 8712
	 Users: 177
	 Items: 1507
	 First Time : 2002-02-28 04:49:07
	 Last Time : 2003-02-28 17:49:50



In [25]:
# train set에 없는 아이템이 val, test기간에 생길 수 있으므로 train data를 기준으로 인덱싱합니다.
id2idx = {item_id : index for index, item_id in enumerate(tr['ItemId'].unique())}

def indexing(df, id2idx):
    df['item_idx'] = df['ItemId'].map(lambda x: id2idx.get(x, -1))  # id2idx에 없는 아이템은 모르는 값(-1) 처리 해줍니다.
    return df

tr = indexing(tr, id2idx)
val = indexing(val, id2idx)
test = indexing(test, id2idx)

In [26]:
save_path = data_path / 'processed'
save_path.mkdir(parents=True, exist_ok=True)

tr.to_pickle(save_path / 'train.pkl')
val.to_pickle(save_path / 'valid.pkl')
test.to_pickle(save_path / 'test.pkl')

In [27]:
class SessionDataset:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, data):
        self.df = data
        self.click_offsets = self.get_click_offsets()
        self.session_idx = np.arange(self.df['UserId'].nunique())  # indexing to UserId

    def get_click_offsets(self):
        """
        Return the indexes of the first click of each session IDs,
        """
        offsets = np.zeros(self.df['UserId'].nunique() + 1, dtype=np.int32)
        offsets[1:] = self.df.groupby('UserId').size().cumsum()
        return offsets

In [28]:
tr_dataset = SessionDataset(tr)
tr_dataset.df.head(10)

,UserId,ItemId,Rating,Time,item_idx
13366,104,2858,5,2002-03-13 21:41:16,0
13338,104,3155,2,2002-03-13 21:41:27,1
13349,104,2707,3,2002-03-13 21:41:27,2
13330,104,2997,4,2002-03-13 21:41:40,3
13368,104,3593,1,2002-03-13 21:41:40,4
13331,104,3005,3,2002-03-13 21:41:50,5
13352,104,3298,3,2002-03-13 21:41:50,6
13364,104,2771,2,2002-03-13 21:41:50,7
13373,104,2976,3,2002-03-13 21:41:50,8
13374,104,3786,2,2002-03-13 21:42:00,9


In [29]:
tr_dataset.click_offsets

array([   0,   12,   17,   19,   40,   46,   63,   74,  123,  135,  216,
        251,  271,  275,  312,  318,  339,  449,  466,  481,  483,  532,
        539,  543,  686,  692,  732,  736,  738,  754,  757,  798,  808,
        817,  823,  947,  955,  958,  972,  996,  999, 1005, 1014, 1022,
       1025, 1031, 1056, 1155, 1190, 1238, 1241, 1302, 1315, 1379, 1403,
       1416, 1421, 1431, 1495, 1662, 1763, 1819, 1821, 1850, 1887, 1970,
       1986, 1992, 2018, 2023, 2025, 2035, 2038, 2045, 2057, 2120, 2181,
       2198, 2220, 2233, 2235, 2335, 2373, 2403, 2416, 2440, 2446, 2585,
       2649, 2659, 2741, 2746, 2754, 2760, 2767, 2786, 2815, 2818, 2892,
       2961, 2967, 2983, 3041, 3090, 3101, 3104, 3130, 3142, 3177, 3180,
       3228, 3232, 3254, 3257, 3268, 3283, 3407, 3434, 3436, 3439, 3453,
       3537, 3592, 3605, 3607, 3631, 3714, 3914, 3938, 4048, 4062, 4065,
       4075, 4081, 4089, 4094, 4100, 4117, 4119, 4148, 4178, 4197, 4199,
       4277, 4431, 4469, 4472, 4486, 4525, 4567, 45

In [30]:
tr_dataset.session_idx

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [31]:
class SessionDataLoader:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, dataset: SessionDataset, batch_size=50):
        self.dataset = dataset
        self.batch_size = batch_size

    def __iter__(self):
        """ Returns the iterator for producing session-parallel training mini-batches.
        Yields:
            input (B,):  Item indices that will be encoded as one-hot vectors later.
            target (B,): a Variable that stores the target item indices
            masks: Numpy array indicating the positions of the sessions to be terminated
        """

        start, end, mask, last_session, finished = self.initialize()  # initialize 메소드에서 확인해주세요.
        """
        start : Index Where Session Start
        end : Index Where Session End
        mask : indicator for the sessions to be terminated
        """

        while not finished:
            min_len = (end - start).min() - 1  # Shortest Length Among Sessions
            for i in range(min_len):
                # Build inputs & targets
                inp = self.dataset.df['item_idx'].values[start + i]
                target = self.dataset.df['item_idx'].values[start + i + 1]
                yield inp, target, mask

            start, end, mask, last_session, finished = self.update_status(start, end, min_len, last_session, finished)

    def initialize(self):
        first_iters = np.arange(self.batch_size)    # 첫 배치에 사용할 세션 Index를 가져옵니다.
        last_session = self.batch_size - 1    # 마지막으로 다루고 있는 세션 Index를 저장해둡니다.
        start = self.dataset.click_offsets[self.dataset.session_idx[first_iters]]       # data 상에서 session이 시작된 위치를 가져옵니다.
        end = self.dataset.click_offsets[self.dataset.session_idx[first_iters] + 1]  # session이 끝난 위치 바로 다음 위치를 가져옵니다.
        mask = np.array([])   # session의 모든 아이템을 다 돌은 경우 mask에 추가해줄 것입니다.
        finished = False         # data를 전부 돌았는지 기록하기 위한 변수입니다.
        return start, end, mask, last_session, finished

    def update_status(self, start: np.ndarray, end: np.ndarray, min_len: int, last_session: int, finished: bool):  
        # 다음 배치 데이터를 생성하기 위해 상태를 update합니다.
        
        start += min_len   # __iter__에서 min_len 만큼 for문을 돌았으므로 start를 min_len 만큼 더해줍니다.
        mask = np.arange(self.batch_size)[(end - start) == 1]  
        # end는 다음 세션이 시작되는 위치인데 start와 한 칸 차이난다는 것은 session이 끝났다는 뜻입니다. mask에 기록해줍니다.

        for i, idx in enumerate(mask, start=1):  # mask에 추가된 세션 개수만큼 새로운 세션을 돌것입니다.
            new_session = last_session + i  
            if new_session > self.dataset.session_idx[-1]:  # 만약 새로운 세션이 마지막 세션 index보다 크다면 모든 학습데이터를 돈 것입니다.
                finished = True
                break
            # update the next starting/ending point
            start[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session]]     # 종료된 세션 대신 새로운 세션의 시작점을 기록합니다.
            end[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session] + 1]

        last_session += len(mask)  # 마지막 세션의 위치를 기록해둡니다.
        return start, end, mask, last_session, finished

In [32]:
tr_data_loader = SessionDataLoader(tr_dataset, batch_size=4)
tr_dataset.df.head(10)

,UserId,ItemId,Rating,Time,item_idx
13366,104,2858,5,2002-03-13 21:41:16,0
13338,104,3155,2,2002-03-13 21:41:27,1
13349,104,2707,3,2002-03-13 21:41:27,2
13330,104,2997,4,2002-03-13 21:41:40,3
13368,104,3593,1,2002-03-13 21:41:40,4
13331,104,3005,3,2002-03-13 21:41:50,5
13352,104,3298,3,2002-03-13 21:41:50,6
13364,104,2771,2,2002-03-13 21:41:50,7
13373,104,2976,3,2002-03-13 21:41:50,8
13374,104,3786,2,2002-03-13 21:42:00,9


In [33]:
iter_ex = iter(tr_data_loader)

In [34]:
inputs, labels, mask =  next(iter_ex)
print(f'Model Input Item Idx are : {inputs}')
print(f'Label Item Idx are : {"":5} {labels}')
print(f'Previous Masked Input Idx are {mask}')

Model Input Item Idx are : [ 0 12 17 19]
Label Item Idx are :       [ 1 13 18 20]
Previous Masked Input Idx are []


In [35]:
def mrr_k(pred, truth: int, k: int):
    indexing = np.where(pred[:k] == truth)[0]
    if len(indexing) > 0:
        return 1 / (indexing[0] + 1)
    else:
        return 0


def recall_k(pred, truth: int, k: int) -> int:
    answer = truth in pred[:k]
    return int(answer)

In [36]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, GRU
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

In [37]:
def create_model(args):
    inputs = Input(batch_shape=(args.batch_size, 1, args.num_items))
    gru, _ = GRU(args.hsz, stateful=True, return_state=True, name='GRU')(inputs)
    dropout = Dropout(args.drop_rate)(gru)
    predictions = Dense(args.num_items, activation='softmax')(dropout)
    model = Model(inputs=inputs, outputs=[predictions])
    model.compile(loss=categorical_crossentropy, optimizer=Adam(args.lr), metrics=['accuracy'])
    model.summary()
    return model

In [38]:
class Args:
    def __init__(self, tr, val, test, batch_size, hsz, drop_rate, lr, epochs, k):
        self.tr = tr
        self.val = val
        self.test = test
        self.num_items = tr['ItemId'].nunique()
        self.num_sessions = tr['UserId'].nunique()
        self.batch_size = batch_size
        self.hsz = hsz
        self.drop_rate = drop_rate
        self.lr = lr
        self.epochs = epochs
        self.k = k



In [39]:
#하이퍼 파라미터 설정부분
args = Args(tr, val, test, batch_size=64, hsz=50, drop_rate=0.5, lr=0.01, epochs=10, k=13)

In [40]:
model = create_model(args)



Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(64, 1, 1463)]           0         
_________________________________________________________________
GRU (GRU)                    [(64, 50), (64, 50)]      227250    
_________________________________________________________________
dropout (Dropout)            (64, 50)                  0         
_________________________________________________________________
dense (Dense)                (64, 1463)                74613     
Total params: 301,863
Trainable params: 301,863
Non-trainable params: 0
_________________________________________________________________


2021-12-15 19:46:06.984304: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-15 19:46:06.984433: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [41]:
# train 셋으로 학습하면서 valid 셋으로 검증합니다.
def train_model(model, args):
    train_dataset = SessionDataset(args.tr)
    train_loader = SessionDataLoader(train_dataset, batch_size=args.batch_size)

    for epoch in range(1, args.epochs + 1):
        total_step = len(args.tr) - args.tr['UserId'].nunique()
        tr_loader = tqdm(train_loader, total=total_step // args.batch_size, desc='Train', mininterval=1)
        for feat, target, mask in tr_loader:
            reset_hidden_states(model, mask)  # 종료된 session은 hidden_state를 초기화합니다. 아래 메서드에서 확인해주세요.

            input_ohe = to_categorical(feat, num_classes=args.num_items)
            input_ohe = np.expand_dims(input_ohe, axis=1)
            target_ohe = to_categorical(target, num_classes=args.num_items)

            result = model.train_on_batch(input_ohe, target_ohe)
            tr_loader.set_postfix(train_loss=result[0], accuracy = result[1])

        val_recall, val_mrr = get_metrics(args.val, model, args, args.k)  # valid set에 대해 검증합니다.

        print(f"\t - Recall@{args.k} epoch {epoch}: {val_recall:3f}")
        print(f"\t - MRR@{args.k}    epoch {epoch}: {val_mrr:3f}\n")


def reset_hidden_states(model, mask):
    gru_layer = model.get_layer(name='GRU')  # model에서 gru layer를 가져옵니다.
    hidden_states = gru_layer.states[0].numpy()  # gru_layer의 parameter를 가져옵니다.
    for elt in mask:  # mask된 인덱스 즉, 종료된 세션의 인덱스를 돌면서
        hidden_states[elt, :] = 0  # parameter를 초기화 합니다.
    gru_layer.reset_states(states=hidden_states)


def get_metrics(data, model, args, k: int):  # valid셋과 test셋을 평가하는 코드입니다. 
                                             # train과 거의 같지만 mrr, recall을 구하는 라인이 있습니다.
    dataset = SessionDataset(data)
    loader = SessionDataLoader(dataset, batch_size=args.batch_size)
    recall_list, mrr_list = [], []

    total_step = len(data) - data['UserId'].nunique()
    for inputs, label, mask in tqdm(loader, total=total_step // args.batch_size, desc='Evaluation', mininterval=1):
        reset_hidden_states(model, mask)
        input_ohe = to_categorical(inputs, num_classes=args.num_items)
        input_ohe = np.expand_dims(input_ohe, axis=1)

        pred = model.predict(input_ohe, batch_size=args.batch_size)
        pred_arg = tf.argsort(pred, direction='DESCENDING')  # softmax 값이 큰 순서대로 sorting 합니다.

        length = len(inputs)
        recall_list.extend([recall_k(pred_arg[i], label[i], k) for i in range(length)])
        mrr_list.extend([mrr_k(pred_arg[i], label[i], k) for i in range(length)])

    recall, mrr = np.mean(recall_list), np.mean(mrr_list)
    return recall, mrr

In [42]:
train_model(model, args)

Train:   0%|          | 0/99 [00:00<?, ?it/s]2021-12-15 19:46:20.963688: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-15 19:46:20.965953: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-15 19:46:20.967521: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-15 19:46:21.062914: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-15 19:46:21.573165: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
Evaluation:   0%|          | 0/48 [00:00<?, ?it/s]2021-12-15 19:46:23.666415: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-

	 - Recall@13 epoch 1: 0.057292
	 - MRR@13    epoch 1: 0.010793



Evaluation:  19%|█▉        | 9/48 [00:00<00:03, 12.95it/s]


	 - Recall@13 epoch 2: 0.032986
	 - MRR@13    epoch 2: 0.010548



Evaluation:  19%|█▉        | 9/48 [00:00<00:02, 13.49it/s]


...

	 - Recall@13 epoch 8: 0.045139
	 - MRR@13    epoch 8: 0.023339



Evaluation:  19%|█▉        | 9/48 [00:00<00:02, 13.70it/s]


	 - Recall@13 epoch 9: 0.045139
	 - MRR@13    epoch 9: 0.022478



Evaluation:  19%|█▉        | 9/48 [00:00<00:02, 13.61it/s]

	 - Recall@13 epoch 10: 0.048611
	 - MRR@13    epoch 10: 0.020450



In [43]:
# Inference
def test_model(model, args, test):
    test_recall, test_mrr = get_metrics(test, model, args, 20)
    print(f"\t - Recall@{args.k}: {test_recall:3f}")
    print(f"\t - MRR@{args.k}: {test_mrr:3f}\n")

test_model(model, args, test)

Evaluation:  56%|█████▋    | 75/133 [00:07<00:05, 10.10it/s]

	 - Recall@13: 0.034167
	 - MRR@13: 0.012079



In [44]:
args = Args(tr, val, test, batch_size=64, hsz=50, drop_rate=0.1, lr=0.01, epochs=5, k=15)
model = create_model(args)
train_model(model, args)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(64, 1, 1463)]           0         
_________________________________________________________________
GRU (GRU)                    [(64, 50), (64, 50)]      227250    
_________________________________________________________________
dropout_1 (Dropout)          (64, 50)                  0         
_________________________________________________________________
dense_1 (Dense)              (64, 1463)                74613     
Total params: 301,863
Trainable params: 301,863
Non-trainable params: 0
_________________________________________________________________


Train:   0%|          | 0/99 [00:00<?, ?it/s]2021-12-15 19:46:47.314737: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-15 19:46:47.363612: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-15 19:46:47.390524: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
Evaluation:   0%|          | 0/48 [00:00<?, ?it/s]2021-12-15 19:46:48.317125: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-15 19:46:48.331940: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
Evaluation:  19%|█▉        | 9/48 [00:00<00:03, 11.26it/s]


	 - Recall@15 epoch 1: 0.050347
	 - MRR@15    epoch 1: 0.011691



Evaluation:  19%|█▉        | 9/48 [00:00<00:03, 12.50it/s]


	 - Recall@15 epoch 2: 0.043403
	 - MRR@15    epoch 2: 0.009782



...

	 - Recall@15 epoch 3: 0.052083
	 - MRR@15    epoch 3: 0.019324



Evaluation:  19%|█▉        | 9/48 [00:00<00:03, 12.42it/s]


	 - Recall@15 epoch 4: 0.062500
	 - MRR@15    epoch 4: 0.023013



Evaluation:  19%|█▉        | 9/48 [00:00<00:03, 12.78it/s]

	 - Recall@15 epoch 5: 0.060764
	 - MRR@15    epoch 5: 0.023878



In [45]:
test_model(model, args, test)

Evaluation:  56%|█████▋    | 75/133 [00:07<00:05, 10.13it/s]

	 - Recall@15: 0.038125
	 - MRR@15: 0.012284



# 세션정의, 모델구조, 하이퍼파라미터 등을 변경해서 실험하여 Recall, MRR 등의 변화추이를 관찰하였다.

In [46]:
args = Args(tr, val, test, batch_size=64, hsz=50, drop_rate=0.2, lr=0.01, epochs=10, k=20)
model = create_model(args)

train_model(model, args)


test_model(model, args, test)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(64, 1, 1463)]           0         
_________________________________________________________________
GRU (GRU)                    [(64, 50), (64, 50)]      227250    
_________________________________________________________________
dropout_2 (Dropout)          (64, 50)                  0         
_________________________________________________________________
dense_2 (Dense)              (64, 1463)                74613     
Total params: 301,863
Trainable params: 301,863
Non-trainable params: 0
_________________________________________________________________


Train:   0%|          | 0/99 [00:00<?, ?it/s]2021-12-15 19:57:50.624651: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-15 19:57:50.677701: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-15 19:57:50.704436: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
Evaluation:   0%|          | 0/48 [00:00<?, ?it/s]2021-12-15 19:57:51.872029: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-15 19:57:51.889445: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
Evaluation:  19%|█▉        | 9/48 [00:01<00:04,  8.98it/s]


	 - Recall@20 epoch 1: 0.074653
	 - MRR@20    epoch 1: 0.014848



Evaluation:  19%|█▉        | 9/48 [00:00<00:03, 10.48it/s]


	 - Recall@20 epoch 2: 0.057292
	 - MRR@20    epoch 2: 0.012699



...

	 - Recall@20 epoch 9: 0.057292
	 - MRR@20    epoch 9: 0.019492



Evaluation:  19%|█▉        | 9/48 [00:00<00:03, 10.46it/s]


	 - Recall@20 epoch 10: 0.055556
	 - MRR@20    epoch 10: 0.017649



Evaluation:  56%|█████▋    | 75/133 [00:07<00:05, 10.14it/s]

	 - Recall@20: 0.033333
	 - MRR@20: 0.011742

